In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertModel
import transformers
import torch
import os
import nltk
import re

from tqdm import tqdm
tqdm.pandas()

In [2]:
df = pd.read_excel(r"C:\Users\user\Desktop\학술대회 2차\데이터\관세용어사전_전처리.xlsx", index_col = 0)

In [3]:
df = df.dropna(axis=0)

In [4]:
# 문자열 길이가 개 이상만 뽑
df = df[df['단어 설명'].str.len() > 10]

In [24]:
df = df[:600]

In [25]:
df = df.reset_index(drop=True)

In [26]:
df

,단어이름,단어 설명
0,인터넷 네트워크 정보센터,인터넷 관련 자원과 정보의 관리를 행하는 기관 각국에 네트워크 정보 센터가 있으며 ...
1,현실전손,절대전손 이라고도 하며 화물이 현실로 전손된 경우 또는 그 점유가 박탈되어 회복할 ...
2,국경 내 장벽,관세나 비관세장벽과 같은 국경 간 장벽은 아니나 차적으로 국내경제를 위해 채택하지만...
3,내항능력,선박이 그 운항을 마치기 위하여 선체 기관 속구를 완비하고 필요한 승무원을 배치하고...
4,간이 입출항절차,외국무역선기이 개항에 입항하여 물품을 하역하지 않고 입항한 때로부터 시간이 내출항하...
...,...,...
595,일반적요,선박회사 측에서의 화물의 수도는 화물의 내용을 문제로 하지 않고 외장 상태만을 보고...
596,포장비,수출포장 에 소요되는 비용으로서 조건으로 수출하는 경우에는 수출자가 부담
597,가격증감약관,라고 부르며 물가나 환율의 변동에 따라 거래계약가격 공사 청부금액 노동자의 임금 등...
598,예약 선박 항공기,선박 또는 항공사의 대리인과 화물선적을 위해 계약하는 것


In [27]:
df_len = df.iloc[df['단어 설명'].str.len().sort_values(ascending=False).index]

In [28]:
df_len = df_len.reset_index(drop=True)

In [29]:
len(df_len["단어 설명"][0])

595

In [30]:
len(df['단어 설명'][0])

183

In [31]:
df_len.to_excel(r"C:\Users\user\Desktop\학술대회 2차\데이터\관세용어사전_길이순.xlsx")

# 단어 순서대로 나열해보기(비교하기위해)

In [65]:
df = pd.read_excel(r"C:\Users\user\Desktop\학술대회 2차\데이터\관세용어사전_길이순.xlsx")

In [66]:
df_high = df[:200]
df_middle = df[200:400]
df_low = df[400:600]

df_high,df_middle,df_low 별 단어 길이확인

In [70]:
df_high = df_high.iloc[df_high['단어이름'].str.len().sort_values(ascending=False).index]
df_middle = df_middle.iloc[df_high['단어이름'].str.len().sort_values(ascending=False).index]
df_low = df_low.iloc[df_high['단어이름'].str.len().sort_values(ascending=False).index]

In [71]:
df_high = df_high.reset_index(drop=True)
df_middle = df_middle.reset_index(drop=True)
df_low = df_low.reset_index(drop=True)

In [72]:
df_high

,Unnamed: 0,단어이름,단어 설명
0,73,쿼터내물량 세율 시장접근물량세율 추천물량세율 쿼터외물량 세율 시장접근초과물량세율 비...,기준에 의하여 실행세율을 계산하는 경우 가 설정된 품목은 가 실행세율로 계산되고 있...
1,121,통일 상품명 및 부호체계에 관한 국제협약,호와 소호 및 관련코드 부류소호의 주와 해석에관한일반통칙 등을 포함한 품목분류표를 ...
2,87,하우스 투 하우스 주소지에서 주소지까지,일방주소지에서 타방주소지까지 내용물의 운송을 위해 화주와 수령자가 사용할 수 있도록...
3,146,기초원재료납세증명 분할증명서 기납분증,내국신용장등으로 거래된 경우로서 기초원재료납세증명서가 발급된 물품을 공급받은 업체가...
4,54,화환신용장에 관한 통일규칙 및 관례,의 약칭이며 신용장 통일규칙이라고 함 국제적으로 이용되고 있는 신용장의 해석 취급 ...
...,...,...,...
195,76,좌초,화물을 실은 선박이 수심이 얕은 암초에 걸려서 단시간에 끌어 내릴 수 없는 상태에 ...
196,17,불착,라고 약칭함 보험상의 는 상자 포대 드럼통 등을 개수로 셀 수 있는 화물이 포장된 ...
197,139,칼라,금융계약상의 이자율 또는 어떤 물건의 가격에 있어서 최고상한과 최저하한을 정하고 있...
198,70,특허,기술적 창작에 대한 독점 배타적 권리 법률에 의해 특권 재산권 또는 권한을 인 이상...


In [73]:
import pandas as pd

In [74]:
# 200개 중 상위 10%
h_h = pd.DataFrame(df_high['Unnamed: 0'][:20])
m_h = pd.DataFrame(df_middle['Unnamed: 0'][:20])
l_h = pd.DataFrame(df_low['Unnamed: 0'][:20])

In [75]:
vocab_high = pd.concat([h_h, m_h, l_h], axis=1)
vocab_high.columns = ["len_high","len_middle","len_low"]

In [76]:
vocab_high

,len_high,len_middle,len_low
0,73,273,473
1,121,321,521
2,87,287,487
3,146,346,546
4,54,254,454
5,49,249,449
6,166,366,566
7,27,227,427
8,22,222,422
9,194,394,594


In [77]:
# 200개 중 중위 10%
h_m = pd.DataFrame(df_high['Unnamed: 0'][91:111])
m_m = pd.DataFrame(df_middle['Unnamed: 0'][91:111])
l_m = pd.DataFrame(df_low['Unnamed: 0'][91:111])

In [79]:
vocab_middle = pd.concat([h_m, m_m, l_m], axis=1)
vocab_middle.columns = ["len_high","len_middle","len_low"]

In [80]:
vocab_middle

,len_high,len_middle,len_low
91,158,374,574
92,110,301,501
93,181,370,570
94,196,288,488
95,113,257,457
96,191,381,581
97,161,396,596
98,162,313,513
99,129,391,591
100,102,201,401


In [82]:
# 200개 중 하위 10%
h_l = pd.DataFrame(df_high['Unnamed: 0'][180:200])
m_l = pd.DataFrame(df_middle['Unnamed: 0'][180:200])
l_l = pd.DataFrame(df_low['Unnamed: 0'][180:200])

In [83]:
vocab_low = pd.concat([h_l, m_l, l_l], axis=1)
vocab_low.columns = ["len_high","len_middle","len_low"]

In [84]:
vocab_low

,len_high,len_middle,len_low
180,21,221,421
181,132,332,532
182,143,343,543
183,55,382,582
184,182,311,511
185,142,342,542
186,9,209,409
187,4,204,404
188,69,269,469
189,86,286,486


In [85]:
vocab_high.to_excel(r"C:\Users\user\Desktop\학술대회 2차\데이터\테스트\vocab_high.xlsx")
vocab_middle.to_excel(r"C:\Users\user\Desktop\학술대회 2차\데이터\테스트\vocab_middle.xlsx")
vocab_low.to_excel(r"C:\Users\user\Desktop\학술대회 2차\데이터\테스트\vocab_low.xlsx")